In [0]:
%run ../Includes/School-Setup

In [0]:
%sql
drop table if exists enrollments;
create table if not exists enrollments 
select * from parquet.`${dataset.school}/enrollments`

In [0]:
%sql
select * from enrollments

In [0]:
%sql
insert into enrollments select * from parquet.`${dataset.school}/enrollments-new`

In [0]:
%sql
select count(1) from enrollments

In [0]:
%sql
select * from json.`${dataset.school}/students-json-new` where student_id = 'S00244';

In [0]:
%sql
select * from students where student_id in(select student_id from json.`${dataset.school}/students-json-new`)

In [0]:
%sql
select * from students where student_id = 'S00244'

In [0]:
%sql
create or replace temp view students_updates as 
select * from json.`${dataset.school}/students-json-new`;

select * from students_updates;
    

In [0]:
%sql
merge into students c
using students_updates u
on c.student_id = u.student_id
when matched and c.email is null and u.email is not null then 
update set c.email = u.email, c.updated = u.updated
when not matched then insert *;
    


In [0]:
%sql
select * from students where student_id = 'S00244'

In [0]:
%sql
create or replace temp view courses_updates(
  course_id String,
  title String,
  instructor String,
  category String,
  price double)
USING CSV
options (
  path "${dataset.school}/courses-csv-new",
  header = "true",
  delimiter = ";"
);

select * from courses_updates;

In [0]:
%sql
merge into courses c
using courses_updates u
on c.course_id = u.course_id and c.title = u.title
when not matched and u.category = 'Computer Science' then 
insert *

In [0]:
%sql
select distinct course_id from courses order by course_id